In [1]:
import tensorflow as tf
import nltk
import numpy as np
import os
import time
import pandas as pd
import csv
import string
import re

In [2]:
print("Num of GPUs available: ", len(tf.test.gpu_device_name()))

Num of GPUs available:  13


### Something new to try
In the process of uploading up a video game onto the market, one would need to come up with a description for their game.<br>
Using machine learning and a simple Recurrent Neural Network (RNN), we will attempt to create a model to assist in writing a game description based on the genre and prompt given, similar to the GEN AI we have today.

In [3]:
genre = input("Which genre: ")

Which genre:  Action


In [4]:
data = pd.read_csv('games.csv')

In [5]:
data.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


### Dataset
We will filter the dataset to those that are of the genre that we want. We will also remove trim the data a bit by removing games that has low ownership.

In [34]:
data2 = pd.DataFrame(data[["Name","Estimated owners","Genres","About the game"]])
data2 = data2[data2['Genres'].str.contains(genre)==True]
data2 = data2[data2['Estimated owners'].str.contains('0 - 20000')==False]
data2 = data2[data2['Estimated owners'].str.contains('0 - 0')==False]

In [7]:
data2

,Name,Estimated owners,Genres,About the game
9,Deadlings: Rotten Edition,50000 - 100000,"Action,Adventure,Indie",Death is lonely. He has zero friends on his Fa...
15,Alien Breed 3: Descent,200000 - 500000,Action,Alien Breed™ 3: Descent is the final explosive...
21,Project: R.E.B.O.O.T 2,20000 - 50000,"Action,Adventure,Casual,Indie",Meet the continuation of the hardcore platform...
23,Turtle Lu,20000 - 50000,"Action,Adventure,Casual,Indie",Turtle Lu Turtle Lu is an old-school Action-Ad...
39,Vanguard Princess,200000 - 500000,"Action,Casual,Indie",Government experiments on a young girl trigger...
...,...,...,...,...
71524,Beyond Polaris Guardian,20000 - 50000,"Action,Adventure,RPG,Early Access",Military schools serve as a crucial step for r...
71615,Sifu,50000 - 100000,"Action,Indie",Whether you choose to play as a male or a fema...
71635,Papa's Freezeria Deluxe,20000 - 50000,"Action,Casual,Indie,Simulation,Strategy",After starting a relaxing job at the oceanfron...
71652,Historic Fighters,50000 - 100000,"Action,Casual,Indie,Strategy",Historic Fighters is a turn-based game that al...


In [9]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jerra\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [8]:
data2 = data2['About the game'].str.join('')
text = data2.str.cat(sep = ' ')
print(f'Length of text: {len(text)} characters')
print(text[:5000])

Length of text: 9068925 characters
Death is lonely. He has zero friends on his FaceTome account and no one to hang out with. So, in order to feel better he begins “Project Deadlings”. Death buys a factory where he can build his laboratory and begin training a massive army of zombie minions. As the army of Deadlings grows, the mazes of the laboratory become deadlier, loaded with puzzles and death-defying traps. Different Deadlings have their own unique abilities: Bonesack is agile - he can run and jump, Creep can climb on walls and ceilings, Lazybrain treads slowly but carefully and Stencher... well Stencher has gastric problems so he can use his powerful gas clouds to fly. You will have to combine all of these abilities to find your way in Death's Maze. Can you help Death to kill his boredom? Will you be able to navigate all 60+ levels available in Deadlings? Will you complete Project Deadlings, and successfully train all of your zombie minions? Arcade side-scroller with strategy eleme

In [10]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
print(vocab)

3153 unique characters
['\x0c', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x81', '\x85', '\x92', '\x93', '\x94', '\x97', '\xa0', '¡', '£', '¦', '§', '©', '«', '¬', '\xad', '®', '°', '²', '´', '·', '¹', '»', '¼', '½', '¿', 'Á', 'Ä', 'Æ', 'É', 'Ô', '×', 'Ø', 'Û', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'î', 'ï', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ú', 'ü', 'ý', 'ā', 'Č', 'č', 'ę', 'ě', 'ğ', 'į', 'ı', 'ļ', 'ł', 'ń', 'Ō', 'ō', 'ř', 'ś', 'ş', 'ū', 'ƒ', 'ʖ', 'ʼ', '˚', '̀', '́', '̃', '̅', '̈', '̋', '̓', '̔', '̙', '̚', '̝', '̠', '̫', '̬', '̱', '̴', '̵', '̶'

### Processing of the description
After retrieving a long string of game description, we get all the possible characters and vectorise them by converting each character to a numeric representation

In [11]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

D
e
a
t
h
 
i
s
 
l


In [14]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

tf.Tensor(
[b'D' b'e' b'a' b't' b'h' b' ' b'i' b's' b' ' b'l' b'o' b'n' b'e' b'l'
 b'y' b'.' b' ' b'H' b'e' b' ' b'h' b'a' b's' b' ' b'z' b'e' b'r' b'o'
 b' ' b'f' b'r' b'i' b'e' b'n' b'd' b's' b' ' b'o' b'n' b' ' b'h' b'i'
 b's' b' ' b'F' b'a' b'c' b'e' b'T' b'o' b'm' b'e' b' ' b'a' b'c' b'c'
 b'o' b'u' b'n' b't' b' ' b'a' b'n' b'd' b' ' b'n' b'o' b' ' b'o' b'n'
 b'e' b' ' b't' b'o' b' ' b'h' b'a' b'n' b'g' b' ' b'o' b'u' b't' b' '
 b'w' b'i' b't' b'h' b'.' b' ' b'S' b'o' b',' b' ' b'i' b'n' b' ' b'o'
 b'r' b'd' b'e'], shape=(101,), dtype=string)
b'Death is lonely. He has zero friends on his FaceTome account and no one to hang out with. So, in orde'
b'r to feel better he begins \xe2\x80\x9cProject Deadlings\xe2\x80\x9d. Death buys a factory where he can build his laborator'
b'y and begin training a massive army of zombie minions. As the army of Deadlings grows, the mazes of t'
b'he laboratory become deadlier, loaded with puzzles and death-defying traps. Different Deadlings have '
b'th

In [15]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [16]:
split_input_target(list("Action"))

(['A', 'c', 't', 'i', 'o'], ['c', 't', 'i', 'o', 'n'])

In [17]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Death is lonely. He has zero friends on his FaceTome account and no one to hang out with. So, in ord'
Target: b'eath is lonely. He has zero friends on his FaceTome account and no one to hang out with. So, in orde'


### Model
What we would like the model to do is to predict the most probable next character when given a prompt. With this, it will be able to generate text,  character by character, using the dataset that we have trained it on.

We will make use of keras model class. It starts with an embedding input layer that with map the character ID to a vector. Then we have 2 GRU layers followed by a Dense output layer.

In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.gru2 = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      state1 = self.gru1.get_initial_state(x)
      state2 = self.gru2.get_initial_state(x)
    else:
       state1, state2 = states
    x, state1 = self.gru1(x, initial_state=state1, training=training)
    x, state2 = self.gru2(x, initial_state=state2, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, [state1, state2]

    if return_state:
      return x, states
    else:
      return x

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
model.summary()

(64, 100, 3154) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  807424    
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 gru_1 (GRU)                 multiple                  6297600   
                                                                 
 dense (Dense)               multiple                  3232850   
                                                                 
Total params: 14,276,178
Trainable params: 14,276,178
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([2004, 1663, 1957, 1262, 1222, 2724,  593, 1984, 1730,  646,  502,
        213,  905, 2756, 1191, 1286, 1689, 3138, 2910, 2873, 2590,   69,
        404,  149, 2249,  782, 2065, 1127, 2862, 2677,  689, 1859, 1714,
         96,  511, 2839,  827, 2950, 1809,  361, 2868, 2748, 3137, 2257,
       2104,  447, 2567,  676, 3058, 2724,  541, 2113,  795,  140, 2576,
        891,  323, 1090, 1057,  114, 1633,  521,   22, 2266, 3012, 2655,
       2221,  669,  708, 2260, 2740,  166, 2994,  373, 2343, 1495, 2245,
       1057, 2708,  399, 1779, 1161,   72,  528, 2737, 1777,  720, 2736,
        955,  613, 2289,  331,  481, 2237,  953, 1858, 2698, 1303, 1148,
       2133], dtype=int64)

In [26]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 100, 3154)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(8.055787, shape=(), dtype=float32)


3151.9832

In [27]:
model.compile(optimizer='adam', loss=loss)

In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [29]:
EPOCHS = 16
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/16
1402/1402 [==============================] - 140s 96ms/step - loss: 2.5283
Epoch 2/16
1402/1402 [==============================] - 134s 95ms/step - loss: 1.4568
Epoch 3/16
1402/1402 [==============================] - 136s 97ms/step - loss: 1.3465
Epoch 4/16
1402/1402 [==============================] - 135s 96ms/step - loss: 1.2959
Epoch 5/16
1402/1402 [==============================] - 134s 95ms/step - loss: 1.2627
Epoch 6/16
1402/1402 [==============================] - 134s 95ms/step - loss: 1.2373
Epoch 7/16
1402/1402 [==============================] - 134s 95ms/step - loss: 1.2166
Epoch 8/16
1402/1402 [==============================] - 135s 96ms/step - loss: 1.1992
Epoch 9/16
1402/1402 [==============================] - 134s 95ms/step - loss: 1.1844
Epoch 10/16
1402/1402 [==============================] - 133s 95ms/step - loss: 1.1714
Epoch 11/16
1402/1402 [==============================] - 135s 96ms/step - loss: 1.1600
Epoch 12/16
1402/1402 [=============================

In [30]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [31]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
prompt = input("Input prompt: ")
li = list(prompt.split(" "))

Input prompt:  Fight


### Output
After the model has been trained, it is able to generate text by passing in some text and predicting the next character, continuously generating text for us.

In [32]:
start = time.time()
states = None
next_char = tf.constant(li)
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Fight the unique Dimensions rockething about different hordes, and whatever you must restauntly meet other difficulty. VERY UPDATING CONTENT! &amp; YOUTUS There are no rooms we launded away, or else you've made in the PVILE-TOON is created! Team FPS with a very own. Om UP494km2or - Legendary Edition up with additional demon experience combined with movies, combat, extremely different styles of menacing may become asteroids, colors and obstacles, obliterate as leaderboard and experience fournets opponents. Variety Of special attributes, battle mechanics and gravity-five surprises sitting with increasingly item killing attacks on the inventory. An odyscend to earn features, and a kick and hide all knows we find three days such as American swords, Shift - suggest for combat! • Official Gameplay •Definity 8 Unique Design Accompany - 4 fierces after exploding, each one bullet eaca! The game is required! Rogue horror, machine gun and brag-as-bike. Up to 8 bonus levels Adamoniac 15 raiders, 1

### 2nd Attempt
As seen from the above output, it is somewhat able to generate proper sentences, but it still contain random string of numbers and characters, as well as spelling mistakes here and there. So this time, we will clean up the input dataset even more, by removing non-english characters and fixing some spacing issues.

In [35]:
data2 = data2['About the game'].str.join('')
text = data2.str.cat(sep = ' ')
words = set(nltk.corpus.words.words())
printable = set(string.printable)
text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())
text = ''.join(filter(lambda x: x in printable, text))
text = re.sub(r'\s([?.,!"](?:\s|$))', r'\1', text)
text = re.sub(r"\b\s+'\b", r"'", text)
print(f'Length of text: {len(text)} characters')
print(text[:5000])

Length of text: 7163423 characters
Death is lonely. He zero on his account and no one to out with. So, in order to feel better he  Project. Death a factory where he can build his laboratory and begin training a massive army of zombie. As the army of, the of the laboratory become, loaded with and death - traps. Different have their own unique : is agile - he can run and jump, Creep can climb on and, slowly but carefully and ... well gastric so he can use his powerful gas to fly. You will have to combine all of these to find your way in Death ' s Maze. Can you help Death to kill his boredom? Will you be able to navigate all 60 + available in? Will you complete Project, and successfully train all of your zombie? Arcade side - with strategy! Four different of zombie with unique! 70 of pure zombie gore fun! Additional game mode - Nightmare - only for the! What ' s new in the Rotten Edition? 10 brand new bonus 15 new in Nightmare Mode Steam and Full controller support Nightmare mode unlocked

In [36]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

92 unique characters


In [37]:
print(vocab)

[' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


In [38]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
  print(chars_from_ids(seq))
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

D
e
a
t
h
 
i
s
 
l
tf.Tensor(
[b'D' b'e' b'a' b't' b'h' b' ' b'i' b's' b' ' b'l' b'o' b'n' b'e' b'l'
 b'y' b'.' b' ' b'H' b'e' b' ' b'z' b'e' b'r' b'o' b' ' b'o' b'n' b' '
 b'h' b'i' b's' b' ' b'a' b'c' b'c' b'o' b'u' b'n' b't' b' ' b'a' b'n'
 b'd' b' ' b'n' b'o' b' ' b'o' b'n' b'e' b' ' b't' b'o' b' ' b'o' b'u'
 b't' b' ' b'w' b'i' b't' b'h' b'.' b' ' b'S' b'o' b',' b' ' b'i' b'n'
 b' ' b'o' b'r' b'd' b'e' b'r' b' ' b't' b'o' b' ' b'f' b'e' b'e' b'l'
 b' ' b'b' b'e' b't' b't' b'e' b'r' b' ' b'h' b'e' b' ' b' ' b'P' b'r'
 b'o' b'j' b'e'], shape=(101,), dtype=string)
b'Death is lonely. He zero on his account and no one to out with. So, in order to feel better he  Proje'
b'ct. Death a factory where he can build his laboratory and begin training a massive army of zombie. As'
b' the army of, the of the laboratory become, loaded with and death - traps. Different have their own u'
b'nique : is agile - he can run and jump, Creep can climb on and, slowly but carefully and ... well gas'
b'tric

In [39]:
split_input_target(list("Action"))

(['A', 'c', 't', 'i', 'o'], ['c', 't', 'i', 'o', 'n'])

In [40]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Input : b'Death is lonely. He zero on his account and no one to out with. So, in order to feel better he  Proj'
Target: b'eath is lonely. He zero on his account and no one to out with. So, in order to feel better he  Proje'


In [41]:
class MyModel2(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.gru2 = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      state1 = self.gru1.get_initial_state(x)
      state2 = self.gru2.get_initial_state(x)
    else:
       state1, state2 = states
    x, state1 = self.gru1(x, initial_state=state1, training=training)
    x, state2 = self.gru2(x, initial_state=state2, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, [state1, state2]

    if return_state:
      return x, states
    else:
      return x

In [42]:
model = MyModel2(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [43]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 93) # (batch_size, sequence_length, vocab_size)


In [44]:
model.summary()

Model: "my_model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  23808     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 gru_3 (GRU)                 multiple                  6297600   
                                                                 
 dense_1 (Dense)             multiple                  95325     
                                                                 
Total params: 10,355,037
Trainable params: 10,355,037
Non-trainable params: 0
_________________________________________________________________


In [45]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [46]:
sampled_indices

array([22, 77, 34, 86, 61, 89, 28, 55, 53, 17, 81, 69, 13, 41, 35, 27, 13,
       25, 74, 26,  1, 71, 54, 39, 80, 90, 84, 33, 91, 79, 78, 69, 36, 63,
       46, 78, 80, 70, 64, 47, 68, 16, 45, 42,  9, 70, 24, 60, 70, 30, 10,
       73, 82, 28, 20,  6, 65, 73, 73, 67, 24, 63, 23, 24, 83, 91, 48, 42,
       14, 75, 45, 53, 15, 16, 51, 81, 33, 69, 75, 12, 90, 19, 73, 61, 36,
       89, 52, 22, 55,  0, 14, 24, 20,  4, 21, 73, 72, 63, 71, 67],
      dtype=int64)

In [47]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 100, 93)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.5332503, shape=(), dtype=float32)


93.06055

In [48]:
model.compile(optimizer='adam', loss=loss)

In [49]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [50]:
EPOCHS = 16
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/16
1108/1108 [==============================] - 84s 73ms/step - loss: 1.6745
Epoch 2/16
1108/1108 [==============================] - 82s 74ms/step - loss: 1.2610
Epoch 3/16
1108/1108 [==============================] - 81s 73ms/step - loss: 1.1998
Epoch 4/16
1108/1108 [==============================] - 80s 71ms/step - loss: 1.1604
Epoch 5/16
1108/1108 [==============================] - 80s 72ms/step - loss: 1.1289
Epoch 6/16
1108/1108 [==============================] - 78s 70ms/step - loss: 1.1018
Epoch 7/16
1108/1108 [==============================] - 82s 73ms/step - loss: 1.0777
Epoch 8/16
1108/1108 [==============================] - 78s 70ms/step - loss: 1.0569
Epoch 9/16
1108/1108 [==============================] - 78s 69ms/step - loss: 1.0391
Epoch 10/16
1108/1108 [==============================] - 77s 69ms/step - loss: 1.0240
Epoch 11/16
1108/1108 [==============================] - 77s 69ms/step - loss: 1.0119
Epoch 12/16
1108/1108 [==============================] - 78s 69

In [59]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
prompt = input("Input prompt: ")
li = list(prompt.split(" "))

Input prompt:  Fight


In [60]:
start = time.time()
states = None
next_char = tf.constant(li)
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Fight together Fight for super creation of cloud been accessible to with furniture and you can turbo - speed button patience. Pose. Realistically increasing combat equipment, aka The color and like the CLASS Red Hode, Main Master, process, . The last of Us, it ' d break about balance and select the research to hidden and weapon like then use physical attack! Yes, we. , the, rebuilt in fierce, race road supernatural. A web of. You will not choose the the exhaustive rapid - fun, and flowing process, a demonic, you will be able to enjoy the proceeds. There ' s never a new game in main threat, you will be able to show off the you  re or go up against and rain, hard twist on the Family Crystal Cataclysm who you are at will. who are you waiting for you to experience an experience unfold won, as the are skilled and the fourth Oculus of The Last. CRUSH 2CDOW  Extra Control - Fully engage over 10 unique to defeat the energy fell and explore deadly warfare even more in the game! Lobby Get ready 

### Analysis
It does seem to be forming better sentences now, and the mean loss for this model with a cleaned dataset appears to be lower too. To further improve the model in the future, we can perhaps select a more specific set of description data to cater more to the type of games we would like to make, and employ techniques such as applying regularisation, adding more layers to the model and training for more epochs.